In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from geometry_msgs.msg import Twist
import ipywidgets as widgets
from IPython.display import display
from rt2_assignment1.srv import Command
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import ipywidgets as widgets
import time

rospy.init_node('robot_control_node')



Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


In [ ]:

x_linear=0
z_angular=0

a = widgets.FloatSlider(min=0, max=1.0)
display(a)

b = widgets.FloatSlider(min=0, max=1.0)
display(b)


def on_value_change(change):
    """Callback for when the slider of the linear velocity is changed, it updates the variable x_linear"""
    global x_linear
    x_linear= change['new']
    
def on_valueang_change(change): 
    """Callback for when the slider of the angular velocity is changed, it updates the variable z_linear"""
    global z_angular
    z_angular= change['new']

a.observe(on_value_change, names='value')
b.observe(on_valueang_change, names='value')

In [ ]:
ui_client = rospy.ServiceProxy('/user_interface', Command)

button_start = widgets.Button(description="start",layout=Layout(width='auto'))
button_stop = widgets.Button(description="stop when target reached",layout=Layout(width='auto'))
button_stop_now = widgets.Button(description="stop now",layout=Layout(width='auto'))
output = widgets.Output()
display(button_start,button_stop,button_stop_now,output)

#handle changes, the observe method of the widget can be used to register a callback.
def on_button_start_clicked(b):
    """Callback for when the button start is pressed, it sends a request to the /user_interface service"""
    with output:
        print("Starting the robot.")
        ui_client("start")
def on_button_stop_clicked(b):
    """Callback for when the button stop is pressed, it sends a request to the /user_interface service"""
    with output:
        print("Robot will stop when target reached.")
        ui_client("stop")
def on_button_stop_now_clicked(b):
    """Callback for when the button stop now is pressed, it sends a request to the /user_interface service"""
    with output:
        print("Robot stopped.")    
        ui_client("stop_now")
        
button_start.on_click(on_button_start_clicked)
button_stop.on_click(on_button_stop_clicked)
button_stop_now.on_click(on_button_stop_now_clicked)

In [ ]:

cmd_vel_msg = Twist()
cmd_vel_msg.linear.x=0
cmd_vel_msg.angular.z=0

cmd_vel_pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

forward = widgets.Button(description="forward",layout=Layout(width='auto'))
turn_right = widgets.Button(description="turn right",layout=Layout(width='auto'))
turn_left = widgets.Button(description="turn left",layout=Layout(width='auto'))
backward = widgets.Button(description="backward",layout=Layout(width='auto'))
stop = widgets.Button(description="stop",layout=Layout(width='auto'))
output = widgets.Output()
display(forward,turn_right,turn_left,backward,stop,output)

#handle changes, the observe method of the widget can be used to register a callback.
def on_button_forward_clicked(b):
    """Callback for when the button forward is pressed, it sends publish to the /cmd_vel topic"""
    with output:
        global cmd_vel_msg,x_linear,z_angular,ui_client
        ui_client("stop_now")
        time.sleep(1/10)
        print("going straight forward.")
        cmd_vel_msg.linear.x=x_linear
        cmd_vel_msg.angular.z=0
        cmd_vel_pub.publish(cmd_vel_msg)
                           
def on_button_backward_clicked(b):
    """Callback for when the button backward is pressed, it sends publish to the /cmd_vel topic"""
    with output:
        global cmd_vel_msg,x_linear,z_angular,ui_client
        ui_client("stop_now")
        time.sleep(1/10)
        print("going straight backward.")
        cmd_vel_msg.linear.x=-x_linear
        cmd_vel_msg.angular.z=0
        cmd_vel_pub.publish(cmd_vel_msg)                           

def on_button_turn_left_clicked(b):
    """Callback for when the button turn left is pressed, it sends publish to the /cmd_vel topic"""
    with output:
        global cmd_vel_msg,x_linear,z_angular,ui_client
        ui_client("stop_now")
        time.sleep(1/10)
        print("turning left.")
        cmd_vel_msg.linear.x=0
        cmd_vel_msg.angular.z=-z_angular
        cmd_vel_pub.publish(cmd_vel_msg) 
                           
def on_button_turn_right_clicked(b):
    """Callback for when the button turn right is pressed, it sends publish to the /cmd_vel topic"""
    with output:
        global cmd_vel_msg,x_linear,z_angular,ui_client
        ui_client("stop_now")
        time.sleep(1/10)
        print("turning right.")
        cmd_vel_msg.linear.x=0
        cmd_vel_msg.angular.z=z_angular
        cmd_vel_pub.publish(cmd_vel_msg)                          
        
def on_button_stop_clicked(b):
    """Callback for when the button stop is pressed, it sends publish to the /cmd_vel topic"""
    with output:
        global cmd_vel_msg,x_linear,z_angular,ui_client
        ui_client("stop_now")
        time.sleep(1/10)
        print("robot stopped.")
        cmd_vel_msg.linear.x=0
        cmd_vel_msg.angular.z=0
        cmd_vel_pub.publish(cmd_vel_msg)
        
forward.on_click(on_button_forward_clicked)
backward.on_click(on_button_backward_clicked)
turn_left.on_click(on_button_turn_left_clicked)
turn_right.on_click(on_button_turn_right_clicked)  
stop.on_click(on_button_stop_clicked)  
